In [1]:
#using COVIDIncidencePOMDPs
using POMCPOW
using POMDPModelTools
using Distributions
using POMDPSimulators
using POMDPs
using POMDPPolicies
using Random
using ARDESPOT
using ParticleFilters
using Statistics
using UUIDs
using MCTS
using MCVI
using Parameters
using POMCPOW
using Roots
using SpecialFunctions
using QuadGK
using LinearAlgebra

In [9]:
########################################################################
############################## PARAMETERS ##############################
########################################################################

@with_kw struct POMDPParams
    # Params from Ferretti et al
    MEAN_GEN_TIME::Float64 = 5.0
    STD_GEN_TIME::Float64 = 1.9
    MEAN_INC_TIME::Float64 = 1.644
    STD_INC_TIME::Float64 = 0.132
    EXP_GROWTH::Float64 = 0.14
    # Fixed modeling params
    DEATH_RATE::Float64 = 0.001
    POP_SIZE::Float64 = 8e6 
    CASE_THRESH::Float64 = POP_SIZE * 0.0043
    TOTAL_TIME::Int64 = 14 
    HOSP_TIME::Int64 = 8 
    INFECTION_SEED = 70
    INFECTION_MIN = 20
    R0_VARIANCE::Float64 = 0.03
    NUM_LEVELS::Int64 = 5
    # Dynamic modeling params
    TESTING_LAG::Tuple{Int64,Int64}
    PREVALENCE_THRESH::Float64
    OBS_DISP::Float64
    MC_STD::Float64
    FALSE_POS::Float64
    FALSE_NEG::Float64
    # Solver paramaters
    OBS_ROUNDING::Int64 = -1
    TREE_DEPTH::Int64 = 21
    TREE_ITERS::Int64 = 100_000
    NUM_PARTICLES::Int64 = 1000_000
    SOLVER_STEPS::Int64 = 100
    SOLVER::String = "pomcpow"
    DISCOUNT::Float64 = 0.99
    EXPLORATION_COEFF::Float64 = 1.0
    # Reward parameters
    SMOOTHNESS_MULT::Float64 = 0.0 #  * 1{prev = new}
    DROP_MULT::Float64 = 2.5 # * min(0, new - prev)
    PREVALENCE_MULT::Float64 = -1/(800. * POP_SIZE) # I(t) / N * 1/800
    OVERFLOW_REWARD::Float64 = -1e-2 
    # Resampling parameters
    NUM_UNIQUES_THRESH::Int64 = 100_000
    RESAMPLE_MULTIPLIER::Float64 = 5.0
    # MCVI-specific parameters
    OBS_BRANCH::Int64 = 8
    STATES_PER_BELIEF::Int64 = 500
    NUM_PRUNE_OBS::Int64 = 1000
    NUM_EVAL_BELIEF::Int64 = 5000
    NUM_OBS::Int64 = 50
end

const BAD_PARAMS = POMDPParams(TESTING_LAG=(7,10), PREVALENCE_THRESH=0.,
                        R0_VARIANCE=0.03, OBS_DISP=0.15, MC_STD=0.5,FALSE_POS = 0.05, FALSE_NEG = 0.05)

const GOOD_PARAMS = POMDPParams(TESTING_LAG=(4, 5), PREVALENCE_THRESH=0.,
                        R0_VARIANCE=0.03, OBS_DISP=0.05, MC_STD=0.2, FALSE_POS = 0.01, FALSE_NEG = 0.01)

const PARAM_MAPPING = Dict("good" => GOOD_PARAMS,
                           "bad" => BAD_PARAMS)

Dict{String,POMDPParams} with 2 entries:
  "bad"  => POMDPParams(5.0, 1.9, 1.644, 0.132, 0.14, 0.001, 8.0e6, 34400.0, 14…
  "good" => POMDPParams(5.0, 1.9, 1.644, 0.132, 0.14, 0.001, 8.0e6, 34400.0, 14…

In [3]:
########################################################################
########################## UTILITY FUNCTIONS ###########################
########################################################################

function get_weibull_params(mu, sigma)
    """
    Exploiting the identity:
      1/2 log(gamma(1+2/k)) - log(gamma(1+1/k)) = 1/2 log(mu^2 + sig^2) - log(mu)
    """
    f_targ = 0.5 * log(sigma^2 + mu^2) - log(mu)
    f = x -> 0.5 * loggamma(1+2/x) - loggamma(1+1/x) - f_targ
    # Find zero of f(), starting guess for k = 1e-4
    k = Roots.find_zero(f, 1e-4)
    lam = mu / gamma(1 + 1/k)
    return (k, lam)
end

function calc_R0(k, lam, r, T)
    integrand = t -> Distributions.pdf(Weibull(k, lam), t) * exp(-r * t)
    integral, err = QuadGK.quadgk(integrand, 0, T)  
    return 1 / integral
end

function growth_rate_for_R0(k, lam, R0, T)
    r_cont = Roots.find_zero(r -> calc_R0(k, lam, r, T) - R0, 1e-4)
    return exp(r_cont) - 1
end

function discrete_weibull(k, lam, total_time)
    integrand = t -> Distributions.pdf(Weibull(k, lam), t) 
    return map(t -> QuadGK.quadgk(integrand, t, t+1)[1], (0:total_time-1))
end

function discrete_lognormal(μ,σ,total_time)
    integrand = t -> Distributions.pdf(LogNormal(μ,σ),t)
    return map(t -> QuadGK.quadgk(integrand, t, t+t)[1], (0:total_time-1))
end


discrete_lognormal (generic function with 1 method)

In [21]:
########################################################################
############################# POMDP MODEL ##############################
########################################################################

struct CovidState
   mobility_diffs::Array{Float64}
   incidences::Array{Float64}
   deaths::Float64
   confounds::Array{Float64}
   prev_a::Int64
end

Base.:+(a::CovidState, b::CovidState) = CovidState(a.mobility_diffs + b.mobility_diffs,
                                                    a.incidences + b.incidences,
                                                    a.deaths + b.deaths,
                                                    a.confounds + b.confounds,
                                                    a.prev_a + b.prev_a)
Base.:/(a::CovidState, b::Int64) = CovidState(a.mobility_diffs / b,
                                              a.incidences / b,
                                              a.deaths / b,
                                              a.confounds / b,
                                              a.prev_a / b)

struct CovidPOMDP <: POMDP{CovidState, Int64, Int64}
    base_R0::Float64
    gen_func::Array{Float64}
    inc_func::Array{Float64}
    initial_state::Array{Float64}
    initial_deaths::Float64
    params::POMDPParams
end

struct StateDistribution
    sum_integrand::Float64
    mort_rate::Float64
    base_R0::Float64
    act::Int64
    prev_state::CovidState
    r0_var::Float64
end

######################################
# ------ TRANSITION FUNCTION --------#
######################################
function POMDPModelTools.rand(rng::AbstractRNG, sd::StateDistribution)
    R0_mult = Distributions.Gamma(1/sd.r0_var, sd.r0_var)
    mobility = cumsum(sd.prev_state.mobility_diffs)[sd.act]
    # mobility = sd.prev_state.mobility_diffs[sd.act]
    new_R0 = rand(rng, R0_mult) * sd.base_R0 * mobility
    incidence = rand(rng, Poisson(new_R0 * sd.sum_integrand))
    new_state = sd.prev_state.incidences[2:end]
    
    #Add deaths to the model
    djk = map(x-> rand(rng, Poisson(new_state[x]*sd.mort_rate)),(1:length(new_state)))
    new_deaths = sum(d for d in djk)
    new_deaths += sd.prev_state.deaths
    new_state = new_state .- djk
    append!(new_state, incidence)
    return CovidState(sd.prev_state.mobility_diffs, new_state, new_deaths,
                    sd.prev_state.confounds, sd.act)
end

function tf(m::CovidPOMDP, s::CovidState, a::Int64)
    ϵ_q = .1;
    ϵ_q = 
    int = dot(m.gen_func.*(1 .- ϵ_q .+ ϵ_q.*m.inc_func), reverse(s.incidences .+ m.params.INFECTION_MIN))
    return StateDistribution(int,m.params.DEATH_RATE, m.base_R0, a, s, m.params.R0_VARIANCE)
end
POMDPs.transition(m::CovidPOMDP, s::CovidState, a::Int64)  = tf(m, s, a) 

######################################
# ------ OBSERVATION FUNCTION -------#
######################################
function obs(m::CovidPOMDP, a::Int64, sp::CovidState)
    ϵ_t = sp.confounds[a]
    mu_sym = (1-m.params.FALSE_NEG)*mean(reverse(sp.incidences[1:end-m.params.TESTING_LAG[1]]).*(1.0 .- m.inc_func[m.params.TESTING_LAG[1]+1:end]))
    mu_asym = (1-m.params.FALSE_NEG)*ϵ_t*dot(reverse(sp.incidences[end-m.params.TESTING_LAG[2]:end-m.params.TESTING_LAG[1]]),m.inc_func[m.params.TESTING_LAG[1]+1:m.params.TESTING_LAG[2]+1])
    mu_healthy = m.params.FALSE_POS*ϵ_t*(m.params.TESTING_LAG[2] - m.params.TESTING_LAG[1])*(m.params.POP_SIZE - sp.deaths - sum(sp.incidences))
    mu = mu_sym + mu_asym + mu_healthy
    #mu = mean(sp.incidences[end-m.params.TESTING_LAG[2]:end-m.params.TESTING_LAG[1]]) 
    #mu = mu * sp.confounds[a]
    #p = 1 - mu * m.params.OBS_DISP / (1 + mu * m.params.OBS_DISP) 
    #r = 1 / m.params.OBS_DISP
    p = 1 / (1 + m.params.OBS_DISP)
    r = mu / m.params.OBS_DISP
    return Distributions.NegativeBinomial(r+1e-4, min(p + 1e-4, 1.0-1e-4))
end
POMDPs.observation(m::CovidPOMDP, s::CovidState, a::Int64, sp::CovidState) = obs(m, a, sp)

######################################
# ------ REWARD FUNCTION -------#
######################################
function r(m::CovidPOMDP, s::CovidState, a::Int64, sp::CovidState)
    """
    Reward function:
        Reward = {new} + 1{new == old} + min(0, new - old) 
        - Encourages high new
        - Encourages consistency
        - Highly penalizes big drops
    """
    smoothness_term = m.params.SMOOTHNESS_MULT * abs(a - s.prev_a)
    drop_term = m.params.DROP_MULT * min(0, a - s.prev_a) 
    prevalence_term = m.params.PREVALENCE_MULT * sp.incidences[end]
    rew = a + smoothness_term + drop_term + prevalence_term
    if sum(sp.incidences[end-m.params.HOSP_TIME:end]) > m.params.CASE_THRESH
        rew = m.params.OVERFLOW_REWARD * sp.incidences[end]
    end
    return rew
end
POMDPs.reward(m::CovidPOMDP, s::CovidState, a::Int64, sp::CovidState) = r(m, s, a, sp) 

######################################
# ------ MISCELLANEOUS FUNCTIONS -----#
######################################
POMDPs.initialstate_distribution(m::CovidPOMDP) =
    InitialStateDistribution(m.initial_state, m.initial_deaths, m.params.MC_STD, m.params.NUM_LEVELS)
#POMDPs.actionindex(::CovidPOMDP, a::Array{Int64}) = sum((a .+ 2).*[1 2 4])+1
POMDPs.actionindex(::CovidPOMDP, a::Array{Int64}) = a + 1
POMDPs.discount(m::CovidPOMDP) = m.params.DISCOUNT
A = []
for a in -1:1,b in -1:1, c in -1:1
    push!(A,[a b c])
end
#POMDPs.actions(m::CovidPOMDP) = A
POMDPs.actions(m::CovidPOMDP) = (1:m.params.NUM_LEVELS)

#######################################
# -- FIXED SOLVER (val estimation) -- #
#######################################
mutable struct FixedPolicy{RNG<:AbstractRNG, P<:Union{POMDP,MDP}, U<:Updater} <: Policy
    rng::RNG
    problem::P
    updater::U # set this to use a custom updater, by default it will be a void updater
end
# The constructor below should be used to create the policy so that the action space is initialized correctly
FixedPolicy(problem::Union{POMDP,MDP}; rng=FixedPolicy.GLOBAL_RNG,
             updater=NothingUpdater()) = FixedPolicy(rng, problem, updater)

## policy execution ##
POMDPPolicies.action(policy::FixedPolicy, s) = actions(policy.problem, s)[1] 
POMDPPolicies.action(policy::FixedPolicy, b::Nothing) = actions(policy.problem)[1]
POMDPPolicies.updater(policy::FixedPolicy) = policy.updater

mutable struct FixedSolver <: Solver
    rng::AbstractRNG
end
FixedSolver(;rng=Random.GLOBAL_RNG) = FixedSolver(rng)
POMDPPolicies.solve(solver::FixedSolver, problem::Union{POMDP,MDP}) = FixedPolicy(solver.rng, problem, NothingUpdater())

In [22]:
########################################################################
######################### CUSTOM INIT BELIEF ###########################
########################################################################


struct InitialBeliefDistribution
    incidences::Array{Float64}
    deaths::Float64
    confound_std::Float64
    num_levels::Int64
end

function POMDPModelTools.rand(rng::AbstractRNG, ibd::InitialBeliefDistribution)
    mobility_diffs = rand(rng, Distributions.Dirichlet(ibd.num_levels, 1))
    incidences = ibd.incidences
    deaths = ibd.deaths
    confounds = map(x->rand(rng, LogNormal(0, ibd.confound_std)), (1:ibd.num_levels))
    #confounds = ones(5)
    prev_a = 1
    return CovidState(mobility_diffs, incidences, deaths, confounds, prev_a)
end

### Custom Initial State Distribution
struct InitialStateDistribution
    incidences::Array{Float64}
    deaths::Float64
    confound_std::Float64
    num_levels::Int64
end

function POMDPModelTools.rand(rng::AbstractRNG, isd::InitialStateDistribution)
    """
    gaps = rand(rng, Distributions.Dirichlet(4, 0.7)) * 0.7
    prepend!(gaps, 0.3)
    mobility_diffs = cumsum(gaps)
    """
    mobility_diffs = [0.3, 0.15, 0.15, 0.1, 0.3]
    incidences = isd.incidences
    deaths = isd.deaths
    confounds = LinRange(0.0,0.1,5)
    #confounds = map(x->rand(rng, LogNormal(0, isd.confound_std)), (1:isd.num_levels))
    prev_a = isd.num_levels
    return CovidState(mobility_diffs, incidences, deaths, confounds, prev_a)
end

function ParticleFilters.resample(r, bp::WeightedParticleBelief, pm::CovidPOMDP,
                                  rm::CovidPOMDP, b, a, o, rng)
    # Run the normal resampling procedure
    ps = Array{CovidState}(undef, r.n)
    ws = Array{Float64}(undef, r.n)
    mobility_diffs::Array{Array{Float64}} = []
    confounders::Array{Array{Float64}} = []
    for bpp in particles(bp)
        push!(mobility_diffs, bpp.mobility_diffs)
        push!(confounders, bpp.confounds)
    end
    unique_mobility_diffs = unique(mobility_diffs)
    unique_confounders = unique(confounders)

    # Logging
    resample_mobs = length(unique_mobility_diffs) < pm.params.NUM_UNIQUES_THRESH
    resample_cfs = length(unique_confounders) < pm.params.NUM_UNIQUES_THRESH / 10

    if !(resample_mobs | resample_cfs)
        return ParticleFilters.resample(LowVarianceResampler(r.n), bp, rng)
    end

    mobility_stdevs = ones(length(unique_mobility_diffs[1])) * 0.1
    confounder_stdevs = ones(length(unique_confounders[1])) * 0.1
    if length(unique_mobility_diffs) > 1
        mobility_stdevs = Statistics.std(unique_mobility_diffs)
    end
    if length(unique_confounders) > 1
        confounder_stdevs = Statistics.std(unique_confounders)
    end

    if resample_mobs
        println("Noising mobilities | std: ", round.(mobility_stdevs, digits=3))
    end
    if resample_cfs
        println("Noising confounds | std: ", round.(confounder_stdevs, digits=3))
    end

    for i in (1:r.n)
        curr_p = particle(bp, i)
        w = weight(bp, i)
        new_mob = curr_p.mobility_diffs
        new_confounds = curr_p.confounds
        if resample_mobs
            mus = clamp.(curr_p.mobility_diffs, 1e-2, 1 - 1e-2) 
            mus = mus / sum(mus) * length(mus) 
            mus = mus * pm.params.RESAMPLE_MULTIPLIER / mean(mobility_stdevs) 
            new_mob = rand(Dirichlet(mus)) 
        end
        if resample_cfs
            mus = curr_p.confounds
            new_confounds = clamp.(rand(MvNormal(mus, max.(confounder_stdevs.^2, 0.01))), 0.5, 2.0)
        end
        ps[i] = CovidState(new_mob, curr_p.incidences, curr_p.deaths, new_confounds, curr_p.prev_a) 
        ws[i] = weight(bp, i)
    end

    new_bp = WeightedParticleBelief(ps, ws)
    return ParticleFilters.resample(LowVarianceResampler(r.n), new_bp, rng)
end

In [24]:
########################################################################
########################### SOLVER TESTING #############################
########################################################################

p = PARAM_MAPPING["bad"]

k, lam = get_weibull_params(p.MEAN_GEN_TIME, p.STD_GEN_TIME)
gen_func = discrete_weibull(k, lam, p.TOTAL_TIME)
inc_func = discrete_lognormal(p.MEAN_INC_TIME,p.STD_INC_TIME,p.TOTAL_TIME)
base_R0 = calc_R0(k, lam, p.EXP_GROWTH, p.TOTAL_TIME)
initial_incidences = ones(Float64, p.TOTAL_TIME) * p.INFECTION_SEED
initial_deaths = 0.0
#initial_belief = InitialBeliefDistribution(initial_incidences, p.NUM_LEVELS)
initial_belief = InitialBeliefDistribution(initial_incidences, initial_deaths, 1.0, p.NUM_LEVELS)

covid_pomdp = CovidPOMDP(base_R0, gen_func, inc_func, initial_incidences, initial_deaths, p)

solver = POMCPOWSolver(max_depth=20, tree_queries=1,
    k_observation=6.0, alpha_observation=1/20., enable_action_pw=false)#,
    #estimate_value=RolloutEstimator(FixedSolver()))

planner = solve(solver, covid_pomdp)
updater = SIRParticleFilter(covid_pomdp, 100)

#stepper = stepthrough(covid_pomdp, planner, updator, initial_belief, "b,s,sp,a,o,r", max_steps=200)

#sim = RolloutSimulator()
sim = HistoryRecorder(max_steps=100)
stepper = simulate(sim, covid_pomdp, planner, updater, initial_belief)

for (s, b, a, r, sp, o) in stepper
    @show s, b, r, sp, o
    @show a
    @show r
end


Noising mobilities | std: [0.165, 0.164, 0.159, 0.155, 0.167]
Noising confounds | std: [2.397, 1.253, 1.365, 4.494, 2.06]
Noising mobilities | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising confounds | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising mobilities | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising confounds | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising mobilities | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising confounds | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising mobilities | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising confounds | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising mobilities | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising confounds | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising mobilities | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising confounds | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising mobilities | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising confounds | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising mobilities | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising confounds | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising mobilities | std: [0.1, 0.1, 0.1, 0.1, 0.1]
No

(s, b, r, sp, o) = (CovidState([0.3, 0.15, 0.15, 0.1, 0.3], [70.0, 70.0, 70.0, 70.0, 70.0, 70.0, 70.0, 70.0, 70.0, 70.0, 70.0, 70.0, 70.0, 70.0], 0.0, [0.0, 0.025, 0.05, 0.07500000000000001, 0.1], 5), 3, 59888, -2.00000001390625, 1)
a = CovidState([0.3, 0.15, 0.15, 0.1, 0.3], [70.0, 70.0, 70.0, 70.0, 69.0, 70.0, 70.0, 70.0, 70.0, 70.0, 70.0, 70.0, 70.0, 89.0], 1.0, [0.0, 0.025, 0.05, 0.07500000000000001, 0.1], 3)
r = 59888
(s, b, r, sp, o) = (CovidState([0.3, 0.15, 0.15, 0.1, 0.3], [70.0, 70.0, 70.0, 70.0, 69.0, 70.0, 70.0, 70.0, 70.0, 70.0, 70.0, 70.0, 70.0, 89.0], 1.0, [0.0, 0.025, 0.05, 0.07500000000000001, 0.1], 3), 4, 89851, 3.99999998, 2)
a = CovidState([0.3, 0.15, 0.15, 0.1, 0.3], [70.0, 70.0, 70.0, 69.0, 70.0, 70.0, 70.0, 70.0, 70.0, 69.0, 70.0, 70.0, 89.0, 128.0], 2.0, [0.0, 0.025, 0.05, 0.07500000000000001, 0.1], 4)
r = 89851
(s, b, r, sp, o) = (CovidState([0.3, 0.15, 0.15, 0.1, 0.3], [70.0, 70.0, 70.0, 69.0, 70.0, 70.0, 70.0, 70.0, 70.0, 69.0, 70.0, 70.0, 89.0, 128.0], 2.0, 

(s, b, r, sp, o) = (CovidState([0.3, 0.15, 0.15, 0.1, 0.3], [68.0, 181.0, 96.0, 140.0, 212.0, 263.0, 133.0, 272.0, 115.0, 83.0, 407.0, 113.0, 282.0, 288.0], 42.0, [0.0, 0.025, 0.05, 0.07500000000000001, 0.1], 4), 3, 60338, 0.49999994859375, 24)
a = CovidState([0.3, 0.15, 0.15, 0.1, 0.3], [181.0, 96.0, 140.0, 212.0, 262.0, 133.0, 272.0, 115.0, 83.0, 407.0, 113.0, 281.0, 288.0, 329.0], 44.0, [0.0, 0.025, 0.05, 0.07500000000000001, 0.1], 3)
r = 60338
(s, b, r, sp, o) = (CovidState([0.3, 0.15, 0.15, 0.1, 0.3], [181.0, 96.0, 140.0, 212.0, 262.0, 133.0, 272.0, 115.0, 83.0, 407.0, 113.0, 281.0, 288.0, 329.0], 44.0, [0.0, 0.025, 0.05, 0.07500000000000001, 0.1], 3), 4, 90542, 3.99999994125, 25)
a = CovidState([0.3, 0.15, 0.15, 0.1, 0.3], [96.0, 140.0, 212.0, 262.0, 133.0, 272.0, 115.0, 82.0, 407.0, 113.0, 280.0, 288.0, 329.0, 376.0], 46.0, [0.0, 0.025, 0.05, 0.07500000000000001, 0.1], 4)
r = 90542
(s, b, r, sp, o) = (CovidState([0.3, 0.15, 0.15, 0.1, 0.3], [96.0, 140.0, 212.0, 262.0, 133.0, 272

(s, b, r, sp, o) = (CovidState([0.3, 0.15, 0.15, 0.1, 0.3], [378.0, 609.0, 504.0, 330.0, 591.0, 240.0, 641.0, 428.0, 616.0, 422.0, 145.0, 820.0, 1093.0, 278.0], 151.0, [0.0, 0.025, 0.05, 0.07500000000000001, 0.1], 1), 2, 30209, 1.99999993703125, 47)
a = CovidState([0.3, 0.15, 0.15, 0.1, 0.3], [609.0, 503.0, 329.0, 590.0, 240.0, 641.0, 427.0, 616.0, 422.0, 145.0, 818.0, 1091.0, 278.0, 403.0], 159.0, [0.0, 0.025, 0.05, 0.07500000000000001, 0.1], 2)
r = 30209
(s, b, r, sp, o) = (CovidState([0.3, 0.15, 0.15, 0.1, 0.3], [609.0, 503.0, 329.0, 590.0, 240.0, 641.0, 427.0, 616.0, 422.0, 145.0, 818.0, 1091.0, 278.0, 403.0], 159.0, [0.0, 0.025, 0.05, 0.07500000000000001, 0.1], 2), 5, 119547, 4.99999986609375, 48)
a = CovidState([0.3, 0.15, 0.15, 0.1, 0.3], [503.0, 328.0, 590.0, 240.0, 641.0, 427.0, 616.0, 422.0, 145.0, 816.0, 1091.0, 278.0, 402.0, 857.0], 163.0, [0.0, 0.025, 0.05, 0.07500000000000001, 0.1], 5)
r = 119547
(s, b, r, sp, o) = (CovidState([0.3, 0.15, 0.15, 0.1, 0.3], [503.0, 328.0, 5

r = 90465
(s, b, r, sp, o) = (CovidState([0.3, 0.15, 0.15, 0.1, 0.3], [539.0, 1844.0, 614.0, 776.0, 2832.0, 1260.0, 1055.0, 2128.0, 1452.0, 2643.0, 1749.0, 3499.0, 3989.0, 2770.0], 427.0, [0.0, 0.025, 0.05, 0.07500000000000001, 0.1], 4), 5, 120318, 4.99999935296875, 69)
a = CovidState([0.3, 0.15, 0.15, 0.1, 0.3], [1843.0, 614.0, 775.0, 2830.0, 1258.0, 1052.0, 2126.0, 1451.0, 2640.0, 1748.0, 3497.0, 3983.0, 2768.0, 4141.0], 453.0, [0.0, 0.025, 0.05, 0.07500000000000001, 0.1], 5)
r = 120318
(s, b, r, sp, o) = (CovidState([0.3, 0.15, 0.15, 0.1, 0.3], [1843.0, 614.0, 775.0, 2830.0, 1258.0, 1052.0, 2126.0, 1451.0, 2640.0, 1748.0, 3497.0, 3983.0, 2768.0, 4141.0], 453.0, [0.0, 0.025, 0.05, 0.07500000000000001, 0.1], 5), 5, 120737, 4.99999947640625, 70)
a = CovidState([0.3, 0.15, 0.15, 0.1, 0.3], [611.0, 775.0, 2827.0, 1256.0, 1052.0, 2125.0, 1449.0, 2633.0, 1747.0, 3496.0, 3980.0, 2763.0, 4137.0, 3351.0], 485.0, [0.0, 0.025, 0.05, 0.07500000000000001, 0.1], 5)
r = 120737
(s, b, r, sp, o) = (C

a = CovidState([0.3, 0.15, 0.15, 0.1, 0.3], [8365.0, 8227.0, 10515.0, 16218.0, 16166.0, 14523.0, 11386.0, 22207.0, 16483.0, 24490.0, 29665.0, 16462.0, 38711.0, 33172.0], 2506.0, [0.0, 0.025, 0.05, 0.07500000000000001, 0.1], 5)
r = 127874
(s, b, r, sp, o) = (CovidState([0.3, 0.15, 0.15, 0.1, 0.3], [8365.0, 8227.0, 10515.0, 16218.0, 16166.0, 14523.0, 11386.0, 22207.0, 16483.0, 24490.0, 29665.0, 16462.0, 38711.0, 33172.0], 2506.0, [0.0, 0.025, 0.05, 0.07500000000000001, 0.1], 5), 5, 128795, -470.2, 90)
a = CovidState([0.3, 0.15, 0.15, 0.1, 0.3], [8222.0, 10503.0, 16201.0, 16149.0, 14504.0, 11369.0, 22184.0, 16450.0, 24465.0, 29633.0, 16456.0, 38677.0, 33140.0, 47020.0], 2778.0, [0.0, 0.025, 0.05, 0.07500000000000001, 0.1], 5)
r = 128795
(s, b, r, sp, o) = (CovidState([0.3, 0.15, 0.15, 0.1, 0.3], [8222.0, 10503.0, 16201.0, 16149.0, 14504.0, 11369.0, 22184.0, 16450.0, 24465.0, 29633.0, 16456.0, 38677.0, 33140.0, 47020.0], 2778.0, [0.0, 0.025, 0.05, 0.07500000000000001, 0.1], 5), 5, 129129, 

In [7]:
AA = []
for a in -1:1,b in -1:1, c in -1:1
push!(AA,[a b c] )
end


In [8]:
AA

27-element Array{Any,1}:
 [-1 -1 -1]
 [-1 -1 0] 
 [-1 -1 1] 
 [-1 0 -1] 
 [-1 0 0]  
 [-1 0 1]  
 [-1 1 -1] 
 [-1 1 0]  
 [-1 1 1]  
 [0 -1 -1] 
 [0 -1 0]  
 [0 -1 1]  
 [0 0 -1]  
 ⋮         
 [0 1 -1]  
 [0 1 0]   
 [0 1 1]   
 [1 -1 -1] 
 [1 -1 0]  
 [1 -1 1]  
 [1 0 -1]  
 [1 0 0]   
 [1 0 1]   
 [1 1 -1]  
 [1 1 0]   
 [1 1 1]   